# Detect Trucks Sentinel-2 - Europe
________________                                            

In [1]:
# load creds
%load_ext dotenv
%dotenv

_______________
## 1 | Setup

In [2]:
# general
import os
import datetime
import pandas as pd
import subprocess
import sys
import shutil
from datetime import date, datetime, timedelta
from glob import glob

# installations
def install_package(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
install_package("OSMPythonTools")
install_package("geocube")

# OSM API
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# EDC Bucket
import s3fs
import requests

# xcube
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube.core.maskset import MaskSet

# spatial
import xarray as xr
import numpy as np
import geopandas as gpd
import geocube
from rasterio import features
from affine import Affine
from shapely import geometry, coords
from shapely.geometry import Polygon, Point
#from osgeo import gdal #, gdal_array, ogr

# plotting
import matplotlib as plt
import IPython.display
%matplotlib inline
figsize = [10,10]

____
## 2 | General Parameters

In [3]:
overwrite_results = True
country = "Germany"
obs_full_threshold = 98 # consider mosaic as full when reached
n_obs = 2

#### Cube

In [4]:
dataset = "S2L2A"
spatial_res = 0.00009 # approx. 10m
bands = ["B02", "B03", "B04", "B08", "B11", "SCL"]
day_bins = "1D" # for cube
tile_size = [512, 512]
minimum_valid_observations = 15 # percent
grid_spacing = 1. # processing grid box size [degree]

#### Directories

In [5]:
dir_main = os.getcwd()
dir_not_commit = os.path.join(dir_main, "not_commit")
dir_ancil = os.path.join(dir_not_commit, "ancillary_data")
dirs = {"dir":dir_main, "dir_not_commit":dir_not_commit, "ancil":dir_ancil, "processing":os.path.join(dir_main, "processing"),
       "processed":os.path.join(dir_not_commit, "processed"), "ancil_roads":os.path.join(dir_ancil, "roads"), "ancil_gadm":os.path.join(dir_ancil, "gadm")}
for directory in list(dirs.values()):
    if not os.path.exists(directory): os.mkdir(directory)

#### Files

In [6]:
files = {"gadm_efta":os.path.join(dirs["ancil_gadm"], "gadm36_0_EU_EFTA.gpkg"), 
         "gadm_europe":os.path.join(dirs["ancil_gadm"], "gadm36_0_europe.gpkg"), 
         "gadm_europe_union":os.path.join(dirs["ancil_gadm"], "gadm0_europe_union.gpkg"),
         "proc_grid":os.path.join(dirs["processing"], "processing_grid_gadm_eu27.geojson" %(grid_spacing))}

#### Temporal

In [7]:
weekdays = ["tuesday", "wednesday", "thursday"] 
target = {"first":datetime(2020, 4, 1), "last":datetime(2020, 6, 21)}
timestamps_sub_period = 1 # how many aggregated timestamps per sub-period
baseline_years = [2017, 2018, 2019] # if processing for a yearly period as in target

#### OSM

In [8]:
osm_key = "highway"
osm_values = ["motorway", "trunk", "primary"]
roads_buffer = 0.00022 # degree, for motorway, the others lower (see OSM methods below)

#### Cloud masking

In [9]:
cloud_masking_thresholds = {"rgb":0.25,
                            "blue_green":0.2,
                            "blue_red":0.2}

______
## 3 | Detection Parameters

Thresholds were determined based on a manually selected dataset of 150 truth points of a test acquisition.

In [10]:
thresholds = {"min_blue":0.06, 
              "min_green":0.04, 
              "min_red":0.04,
              "max_red":0.15,
              "max_green":0.15,
              "max_blue":0.2,
              "max_ndvi":0.5,
              "max_ndwi":0.0001,
              "max_ndsi":0.0001,
              "min_blue_green_ratio":0.03,
              "min_blue_red_ratio":0.05, 
              "max_blue_green_ratio":0.17, 
              "max_blue_red_ratio":0.2}

________________
## 4 | Utils

#### Names

In [11]:
def EPSG_4326(): return "EPSG:4326"
def EPSG_3857(): return "EPSG:3857"
def GEOJSON(): return "GeoJSON"
def GPKG(): return "GPKG"
def GEOJSON_EXT(): return ".geojson"
def GPKG_EXT(): return ".gpkg"
def NC_EXT(): return ".nc"
def BBOX_ID(): return "bbox_id"
def ABS_N_TRUCKS(): return "acquisitions_trucks"
def TRUCKS_VEC(): return "trucks_points"
def TRUCKS_VEC_PHR(): return "trucks_points_placeholder"
def TRUCKS_VEC_MERGED(): return "trucks_points_merged"
def N_OBS(): return "n_observations"
def VALID_PIXELS(): return "valid_pixels"
def TRUCK_COUNT_NORM(): return "truck_count_normalized"
def SUM_TRUCKS_SUB(): return "sum_trucks_sub_period"
def MEAN_TRUCKS(): return "mean_trucks"

def BUCKET_NAME(): return "edc-b62330c3-ed93-4b41-a90f-28f3a73107e9"
def BUCKET_URL(): return "https://edc-b62330c3-ed93-4b41-a90f-28f3a73107e9.s3.eu-central-1.amazonaws.com"

#### EDC BUCKET

In [12]:
s3 = s3fs.S3FileSystem()

#### File names

In [13]:
def fname_osm(directory, bbox_id, osm_key, ext = GPKG_EXT()): return os.path.join(directory, str(bbox_id) + "_" + osm_key + ext)
def construct_fname(dirs_ts, dir_ts_key, bbox_id, ext):
    return os.path.join(dirs_ts[dir_ts_key], dir_ts_key + "_" + BBOX_ID() + str(bbox_id) + ext)
def fname_acquisition_trucks(dirs_ts, bbox_id, date, ext = NC_EXT()): 
    return os.path.join(dirs_ts[ABS_N_TRUCKS()], str(date) + "_" + ABS_N_TRUCKS() + "_" + BBOX_ID() + str(bbox_id) + ext)
def fname_sum_obs(dirs_ts, bbox_id, date, ext = NC_EXT()): 
    return construct_fname(dirs_ts, N_OBS(), str(bbox_id) + "_" + str(date), ext)
def fname_trucks_vec(dirs_ts, bbox_id, date, ext):
    return construct_fname(dirs_ts, TRUCKS_VEC(), str(bbox_id) + "_" + str(date), ext)
def fname_trucks_vec_placeholder(dirs_ts, bbox_id, date, ext = ".txt"):
    return construct_fname(dirs_ts, TRUCKS_VEC(), str(bbox_id) + "_" + str(date), ext)

def sub_period_fnames(dirs_ts, bbox_id, date, ext_arr, ext_vec):
    files = {N_OBS():fname_sum_obs(dirs_ts, bbox_id, date, ext_arr),
             TRUCKS_VEC():fname_trucks_vec(dirs_ts, bbox_id, date, ext_vec),
             TRUCKS_VEC_PHR():fname_trucks_vec_placeholder(dirs_ts, bbox_id, date)}
    return files

#### Directory structure

In [14]:
def make_dirs_ts(dir_ts):
    dir_ts_overall = os.path.join(dir_ts, "overall")
    if not os.path.exists(dir_ts_overall): os.mkdir(dir_ts_overall)
    dirs_ts = {ABS_N_TRUCKS():os.path.join(dir_ts, ABS_N_TRUCKS()),
               N_OBS():os.path.join(dir_ts_overall, N_OBS()),
               TRUCKS_VEC():os.path.join(dir_ts_overall, TRUCKS_VEC()),
               TRUCKS_VEC_MERGED():os.path.join(dir_ts_overall, TRUCKS_VEC_MERGED())}
    for direc in dirs_ts.values():
        if not os.path.exists(direc): os.mkdir(direc)
    return dirs_ts

In [15]:
def delete_recurs(directory, validation):
    if validation == "YES_DELETE_THAT":
        shutil.rmtree(directory)
    else:
        raise Exception("Validation wrong")

#### Processing messages

In [16]:
def start_msg(weekdays, n_boxes, periods, timestamps_sub_period, osm_values, baseline_years, minimum_valid_observations, overwrite_results):
    print("%s\n\nStarting truck detection processing \n%s\n\nWeekdays: \
%s\nNumber of grid cells to process: %s\nNumber of periods: %s\nTimestamps sub-period: \
%s\nOSM roads: %s\nBaseline years: %s\nMinimum valid observations to consider acquisition: \
%s %%\nOverwrite results: %s\n%s\n\n%s" %("="*100, "."*50, weekdays, str(n_boxes), str(len(periods["first"])), 
                                          str(timestamps_sub_period), str(osm_values), str(baseline_years), 
                                          str(minimum_valid_observations), str(overwrite_results), "."*50, "="*100))

#### Processing grid
Create a grid covering Europe.

In [17]:
def make_grid(grid_spacing, files):
    crs = EPSG_4326()
    gadm0_eu_efta = gpd.read_file(files["gadm_efta"])
    xmin,ymin,xmax,ymax = gadm0_eu_efta.total_bounds
    width = xmax - xmin
    height = ymax - ymin
    cols = int((width) / grid_spacing)
    rows = int((height) / grid_spacing)
    box_width = width / cols
    box_height = height / rows
    boxes = []
    for row in range(rows):
        for col in range(cols):
            col_right = col + 1
            row_lower = row + 1
            y_up = ymax-row*box_height
            y_low = ymax-row_lower*box_height
            x_left = xmin+col*box_width
            x_right = xmin+col_right*box_width
            boxes.append(Polygon([(x_left, y_up), 
                                  (x_right, y_up), 
                                  (x_right, y_low), 
                                  (x_left, y_low)]))
    grid = gpd.GeoDataFrame({"geometry":boxes})
    grid.crs = crs
    gadm0_europe = gpd.read_file(files["gadm_europe"])
    gadm0_europe.crs = crs
    file_union = files["gadm_europe_union"]
    if os.path.exists(file_union):
        gadm0_europe_clip_union = gpd.read_file(file_union)
    else:
        gadm0_europe["continent"] = ["EUROPE"] * len(gadm0_europe)
        gadm0_europe_clip = gpd.overlay(gadm0_europe, gpd.GeoDataFrame({"a":[1],"geometry":test}), how = "intersection")
        gadm0_europe_clip_union = gadm0_europe_clip.dissolve(by = "continent")
        gadm0_europe_clip_union.to_file(file_union, driver = GPKG())
    # intersect with gadm for cutting boxes at coast line
    grid_intersect = gpd.overlay(grid, gadm0_europe_clip_union, how = "intersection")
    # get country attributes
    grid_gadm = gpd.sjoin(grid_intersect, gadm0_europe, how = "left", op = "intersects")
    grid_gadm[BBOX_ID()] = range(len(grid_gadm))
    boxes = grid_gadm.geometry.apply(lambda geom : geom.bounds)
    grid_gadm.geometry = [Polygon(geometry.box(b[0], b[1], b[2], b[3])) for b in boxes] # use the light bboxes
    delete_cols = ["GID_0_left", "NAME_0_left", "a", "continent", "index_right"]
    for column in delete_cols:
        if column in grid_gadm.columns: 
            grid_gadm = grid_gadm.drop(column, 1)
    grid_gadm = grid_gadm.rename(columns = {"GID_0_right":"GID_0", "NAME_0_right":"NAME_0"})
    grid_gadm = grid_gadm[grid_gadm["GID_0"] != "RUS"] # do not include
    grid_gadm.to_file(files["proc_grid"], driver = GEOJSON())
    return grid_gadm

#### Vectors

In [18]:
def transform_lat_lon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(polygons, lat, lon, fill=np.nan):
    transform = transform_lat_lon(lat, lon)
    out_shape = (len(lat), len(lon))
    raster = features.rasterize(polygons.geometry, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float)
    return xr.DataArray(raster, coords={"lat":lat, "lon":lon}, dims=("lat", "lon"))

def vec_driver_from_ext(ext):
    return {GPKG_EXT():GPKG(), GEOJSON_EXT():GEOJSON()}[ext]

#### Arrays

In [19]:
# data np array
# lat_lon dict of "lat" and "lon" holding np arrays of coordinates
def create_xr_dataset(data, lat_lon, name):
    lat, lon = "lat", "lon"
    data_array = xr.DataArray(data, coords={lat:lat_lon[lat], lon:lat_lon[lon]}, dims=(lat, lon))
    return xr.Dataset({name:data_array})

# extracts coordinates at value in np array and returns points as GeoDataFrame
# data 2d np array
# match_value Float value in data where point coordinates are extracted
# lon_lat dict of:
### "lon": np array longitude values"
### "lat": np array latitude values"
# crs String EPSG:XXXX
def points_from_np(data, match_value, lon_lat, crs):
    indices = np.argwhere(data == match_value)
    if len(indices) > 0:
        lat_indices = indices[:,[0]]
        lon_indices = indices[:,[1]]
        lat_coords = lon_lat["lat"][lat_indices]
        lon_coords = lon_lat["lon"][lon_indices]
        points = gpd.GeoDataFrame(geometry = gpd.points_from_xy(lon_coords, lat_coords))
        points.crs = crs
        return points
    
def raster_to_points(raster, lon_lat, field_name, crs):
    points_list = []
    match_values = np.unique(raster[(raster != 0) * ~np.isnan(raster)]) # by pixel value
    for x in match_values:
        points = points_from_np(raster, x, lon_lat, crs=crs)
        points[field_name] = [x] * len(points)
        points_list.append(points)
    return gpd.GeoDataFrame(pd.concat(points_list, ignore_index=True))

# take xarray and ensure each value with 1 in data has no neighbor with 1 in an extended 3x3 block. Extended means: horizontally and vertically
# it is also checked for the second-next pixel
# Method checks only surrounding of values equal 1
# arr xarray DataArray with values and lat lon
def filter_spatial_3x3_extended(arr):
    values = arr.values
    lon = arr.lon
    lat = arr.lat
    valid = np.where(arr == 1)
    for y,x in zip(valid[0], valid[1]):
        y_above = y - 1
        y_above_next = y - 2
        x_left = x - 1
        x_right = x + 1
        x_left_next = x - 2
        space_left = x_left >= 0
        space_right = x_right >= 0 and x_right < len(lon)
        space_above = y_above >= 0
        val_left_above = values[y_above, x_left] if space_left and space_above else 0
        val_right_above = values[y_above, x_right] if space_right and space_above else 0
        val_left = values[y, x_left] if space_left else 0
        val_above = values[y_above, x] if space_above else 0
        val_left_next = values[y, x_left_next] if x_left_next >= 0 else 0
        val_above_next = values[y_above_next, x] if y_above_next >= 0 else 0
        # if any of the values left, above and left above has 1 set current value 0
        if (val_left_above + val_right_above + val_left + val_above + val_left_next + val_above_next) >= 1:
            values[y,x] = 0
    arr.values = values
    return arr

def calc_rgb_cloud_mask(band_stack, cloud_masking_thresholds):
    B02, B03, B04 = band_stack.B02, band_stack.B03, band_stack.B04
    c = cloud_masking_thresholds["rgb"]
    clouds_rgb = ((B02 > c) + (B03 > c) + (B04 > c)) >= 1
    # attempt to mask haze without masking out truck pixels (similar! higher blue than red and green)
    blue_green_ratio = (B02-B03) / (B02+B03)
    blue_red_ratio = (B02-B04) / (B02+B04)
    clouds_blue_green = blue_green_ratio > cloud_masking_thresholds["blue_green"]
    clouds_blue_red = blue_red_ratio > cloud_masking_thresholds["blue_red"]
    clouds = (clouds_rgb + clouds_blue_green + clouds_blue_red) >= 1
    return clouds

#### Temporal

In [20]:
# date Datetime object to be checked
# weekday String weekday to be checked against
# returns Boolean if date is weekday
def is_weekday(date_x, weekday):
    if not isinstance(date_x, type(datetime.date)): date_x = pd.to_datetime(date_x).date()
    weekday = weekday.lower()[0:3]
    y, m, d = 2000, 1, 3
    wd = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
    ref = {}
    for i in range(len(wd)): ref[wd[i]] = datetime(y, m, d + i).date()
    return (date_x - ref[weekday]).days % 7 == 0 # check if date is in a sequence of 7

# Calculates the dates of the periods representing a timestamp each
# n_days_sub Integer how many dates shall one timestamp (sub-period) cover
# baseline Dict start and end dates (datetime) overall baseline period
# target Dict start and end dates (datetime) target period
def calc_periods(n_days_sub, baseline, target):
    fst, lst = "first", "last"
    base_first = baseline[fst]
    n_days = target[fst] - base_first
    n_sub = int(n_days.days / n_days_sub) # number of timestamps
    start = [base_first] * n_sub
    start = [start[i] + timedelta(n_days_sub * i) for i in range(len(start))]
    end = [start[i+1] - timedelta(1) for i in range(len(start)-1)]
    end.append(baseline[lst])
    periods = {"ts":range(len(start)), fst:start, lst:end}
    periods["first"].append(target["first"]) # append start date of target period
    periods["last"].append(target["last"]) # append end date of target period
    return periods

# Returns periods equivalent to target period for other years
# target Dict start and end dates (datetime) target period
# years List of int years
# timestamps_sub_period int
def yearly_period_from_target(target, years = [2017, 2018, 2019], timestamps_sub_period = 2):
    n_days_target = (target["last"]-target["first"]).days
    n_days_timestamp = int(n_days_target / timestamps_sub_period)
    trgt_fst = []
    trgt_lst = []
    for i in range(timestamps_sub_period):
        start = target["first"] + timedelta(days = i * (n_days_timestamp + 1))
        trgt_fst.append(start)
        trgt_lst.append(start + timedelta(days = n_days_timestamp))
    start = []
    end = []
    for year in years:
        for fst, lst in zip(trgt_fst, trgt_lst):
            start.append(datetime(year, fst.month, fst.day))
            end.append(datetime(year, lst.month, lst.day))
    appended = [start.append(x) for x in trgt_fst]
    appended = [end.append(x) for x in trgt_lst]
    periods = {"ts":range(len(start)), "first":start, "last":end}
    return periods

# date_str String YYYY-MM-DD
def string_to_datetime(date_str):
    y, m, d = date_str[0:4], date_str[5:7], date_str[-2:]
    if m[0] == "0": m = m[1]
    if d[0] == "0": d = d[1]
    return datetime(int(y), int(m), int(d))

#### OSM data
Utils for retrieving road data from OSM through its API

In [21]:
# re-order bbox from W,S,E,N to S,W,N,E
def convert_bbox_osm(bbox):
    offset = 0.05 # add a buffer to bbox in order to be sure cube is entirely covered
    bbox_osm = [bbox[1], bbox[0], bbox[3], bbox[2]]
    bbox_osm[0] -= offset # min lat
    bbox_osm[1] -= offset # min lon
    bbox_osm[2] += offset # max lat
    bbox_osm[3] += offset # max lon
    return bbox_osm

# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_value String OSM value
# osm_key String OSM key
# element_type List of String
# returns GeoPandasDataFrame
def get_osm(bbox, 
            bbox_id,
            osm_value = "motorway",
            osm_key = "highway", # in OSM 'highway' contains several road types: https://wiki.openstreetmap.org/wiki/Key:highway
            element_type = ["way", "relation"]):
    
    bbox_osm = convert_bbox_osm(bbox)
    quot = '"'
    select = quot+osm_key+quot + '=' + quot+osm_value+quot
    select_link = select.replace(osm_value, osm_value + "_link") # also get road links
    select_junction = select.replace(osm_value, osm_value + "_junction")
    geoms = []
    for selector in [select, select_link, select_junction]:  
        try:
            query = overpassQueryBuilder(bbox=bbox_osm, 
                                         elementType=element_type, 
                                         selector=selector, 
                                         out='body',
                                         includeGeometry=True)
            elements = Overpass().query(query, timeout=60).elements()
            # create multiline of all elements
            if len(elements) > 0:
                for i in range(len(elements)):
                    elem = elements[i]
                    geoms.append(elem.geometry())
        except:
            Warning("Could not retrieve " + select)
    try:
        lines = gpd.GeoDataFrame(crs = EPSG_4326(), geometry = geoms)
        n = len(geoms)
        lines[BBOX_ID()] = [bbox_id]*n
        lines["osm_value"] = [osm_value]*n # add road type
        return lines
    except:
        Warning("Could not merge " + osm_value)
        
# buffer Float road buffer distance [m]
# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_values List of String OSM values
# osm_key String OSM key
# roads_buffer Float buffer width
# dir_write
def get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dir_write):
    fwrite = fname_osm(dir_write, bbox_id, osm_key)
    if not os.path.exists(fwrite):
        roads = []
        has_error = []
        offset = 0.00002
        buffer_dist = "buffer_distance"
        # buffer according to road type
        m,t,p,s,ter = "motorway", "trunk", "primary", "secondary", "tertiary"
        buffers = {m:roads_buffer, t:roads_buffer-offset, p:roads_buffer-(2*offset), s:roads_buffer-(3*offset), ter:roads_buffer-(4*offset)}
        osm_values_int = {m:1, t:2, p:3, s:4, ter:5}
        for osm_value in osm_values:
            try:
                roads_osm = get_osm(bbox = bbox, bbox_id = bbox_id, osm_value = osm_value)
                roads_osm[buffer_dist] = [buffers[osm_value]] * len(roads_osm)
                roads_osm["osm_value_int"] = osm_values_int[osm_value]
                roads.append(roads_osm)
            except:
                has_error.append(1)
                Warning("'get_osm'" + "failed for bbox_id "+ str(bbox_id) + "osm_value " + osm_value + "osm_key" + osm_key)
        if len(roads) > len(has_error):
            roads_merge = gpd.GeoDataFrame(pd.concat(roads, ignore_index=True), crs=roads[0].crs)
            buffered = roads_merge.buffer(distance=roads_merge[buffer_dist])
            roads_merge.geometry = buffered
            roads_merge.to_file(fwrite, driver = GPKG())
    return fwrite

# osm geodataframe of polygons
# reference_raster xarray with lat and lon
def rasterize_osm(osm, reference_raster):
    osm_values = list(set(osm["osm_value"]))
    nan_placeholder = 100
    road_rasters = []
    for osm_value in osm_values:
        osm_subset = osm[osm["osm_value"] == osm_value]
        raster = rasterize(osm_subset, reference_raster.lat, reference_raster.lon)
        cond = np.isfinite(raster)
        raster_osm = np.where(cond, list(osm_subset.osm_value_int)[0], nan_placeholder) # use placeholder instead of nan first
        raster_osm = raster_osm.astype(np.float)
        road_rasters.append(raster_osm)        
    # merge road types in one layer
    road_raster_np = np.array(road_rasters).min(axis=0) # now use the lowest value (highest road level) because some intersect
    road_raster_np[road_raster_np == nan_placeholder] = 0
    return road_raster_np # 0=no_road 1=motorway, 2=trunk, ...

def osm_values_to_name(values):
    mapping = {1:"Motorway", 2:"Trunk", 3:"Primary", 4:"Secondary", 5:"Tertiary"}
    return [mapping[value] for value in values]

_________
## 5 | Truck detection

#### TruckDetector

In [22]:
# TruckDetector detects trucks at acquisition-level
class TruckDetector():   
    def __init__(self, band_stack):
        self.band_stack = band_stack
        is_none = band_stack is None
        self.B02 = None if is_none else band_stack.B02 
        self.B03 = None if is_none else band_stack.B03
        self.B04 = None if is_none else band_stack.B04
        self.B08 = None if is_none else band_stack.B08
        self.B11 = None if is_none else band_stack.B11
        self.no_truck_mask = None
        self.trucks = None
            
    # Calculate a binary mask where pixels that are definitely no trucks are represented as 0.
    # thresholds Dict with at least:
    ### max_ndvi Float above this val: no trucks. For Vegetation
    ### max_ndwi Float above this val: no trucks. For Water
    ### max_ndsi Float above this val: no_trucks. For Snow
    ### min_rgb Float above this val: no_trucks. For dark surfaces, e.g. shadows
    ### max_blue Float above this val: no_trucks
    ### max_green Float above this val: no trucks
    ### max_red Float above this val: no trucks
    ### min_b11 Float below this val: no trucks. For dark surfaces, e.g. shadows
    ### max_b11 Float below this val: no trucks. For bright (sealed) surfaces, e.g. buildings
    def calc_no_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        B08 = self.B08
        B11 = self.B11
        min_blue = thresholds["min_blue"]
        min_green = thresholds["min_green"]
        min_red = thresholds["min_red"]
        max_blue = thresholds["max_blue"]
        max_green = thresholds["max_green"]
        max_red = thresholds["max_red"]
        ndvi_mask = ((B08 - B04) / (B08 + B04)) < thresholds["max_ndvi"]
        ndwi_mask = ((B02 - B11) / (B02 + B11)) < thresholds["max_ndwi"]
        ndsi_mask = ((B03 - B11) / (B03 + B11)) < thresholds["max_ndsi"]
        low_rgb_mask = (B02 > min_blue) * (B03 > min_green) * (B04 > min_red)
        high_rgb_mask = (B02 < max_blue) * (B03 < max_green) * (B04 < max_red)
        self.no_truck_mask = ndvi_mask * ndwi_mask * ndsi_mask * low_rgb_mask * high_rgb_mask
    
    # Calculate a binary mask where trucks are represented as 1 and no trucks as 0.
    # thresholds Dict with at least:
    ### min_green_ratio Float, minimum value of blue-green ratio
    ### min_red_ratio Float, minimum value of blue-red ratio
    def detect_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        bg_ratio = (B02 - B03) / (B02 + B03)
        br_ratio = (B02 - B04) / (B02 + B04)
        bg_ratio_masked = bg_ratio * self.no_truck_mask
        br_ratio_masked = br_ratio * self.no_truck_mask
        bg = bg_ratio_masked > thresholds["min_blue_green_ratio"]
        br = br_ratio_masked > thresholds["min_blue_red_ratio"]
        bg_max = bg_ratio_masked < thresholds["max_blue_green_ratio"]
        br_max = br_ratio_masked < thresholds["max_blue_red_ratio"]
        self.trucks = bg * br * bg_max * br_max
        
    def filter_trucks(self):
        self.trucks = filter_spatial_3x3_extended(self.trucks)

_________
## 6 | Processing classes

#### PeriodProcessor

In [23]:
# PeriodProcessor processes a period of dates, represented in one cube
class PeriodProcessor():
    def __init__(self, start, end, bbox, bbox_id):
        self.start = start
        self.end = end
        self.bbox = bbox
        self.bbox_id = bbox_id
        self.cube = None
        self.dates = None
        self.lon_lat = None
        self.mean_trucks = None
        self.sum_trucks = None
        self.sum_obs = None
        self.trucks_points_result = None
        self.osm_mask = None
        self.used_dates = []
        self.percentage_valid = 0
    
    def get_cube(self, dataset, bands, tile_size, spatial_res, day_bins):
        config = CubeConfig(dataset_name = dataset,
                            band_names = bands,
                            tile_size = tile_size,
                            geometry = self.bbox,
                            spatial_res = spatial_res,
                            time_range = [self.start, self.end])
        cube = open_cube(config)
        self.cube = cube
        self.dates = cube.time.values
        self.lon_lat = {"lon":cube.lon.values, "lat":cube.lat.values}
    
    def calc_osm_mask(self, osm):
        osm_raster = rasterize_osm(osm, self.cube.B02.sel(time = self.dates[0]))
        self.osm_mask = create_xr_dataset(osm_raster, self.lon_lat, "roadmask")
    
    # Add from acquisition methods
    def add_n_observations(self, no_clouds):
        obs = np.where(no_clouds == 1, 1, 0)
        obs[np.isnan(obs)] = 0
        if self.sum_obs is None:
            self.sum_obs = obs.copy()
        else:
            self.sum_obs += obs.copy() # addition
            
    def percentage_covered(self):
        self.sum_obs[self.osm_mask.roadmask.values == 0] = -1
        n_valid = len(self.sum_obs[self.sum_obs >= 1].ravel())
        n_all = len(self.sum_obs[self.sum_obs >= 0].ravel())
        self.percentage_valid = (n_valid / n_all) * 100
        
    def obs_time(self, n_obs):
        n_pixels_roads = len(period.sum_obs[period.sum_obs >= 0])
        n_pixels_obs = len(period.sum_obs[period.sum_obs >= n_obs])
        return (n_pixels_obs / n_pixels_roads) * 100
    
    def add_detections(self, trucks):
        if self.sum_trucks is None:
            self.sum_trucks = trucks.copy()
        else:
            self.sum_trucks += trucks.copy()
    
    def add_acquisition(self, acquisition, date_str):
        self.add_n_observations(acquisition.no_clouds)
        self.add_detections(acquisition.detector.trucks.values)
        self.used_dates.append(date_str)
    
    # Temporal summary methods
    def sum_truck_n(self):
        self.sum_trucks = np.array(self.detections).sum(axis=0)
    
    def sum_observations(self):
        self.sum_obs = np.array(self.n_observations).sum(axis=0)
    
    def mask_sum_obs_to_trucks(self):
        self.sum_obs[self.sum_trucks == 0] = 0
    
    def mean_truck_n(self):
        self.mean_trucks = np.divide(self.sum_trucks, self.sum_obs)
        self.mean_trucks[np.isnan(self.mean_trucks)] = 0
    
    def mask_osm_to_trucks(self, osm_mask):
        self.osm_mask = self.osm_mask.roadmask.values
        self.osm_mask[self.sum_trucks == 0] = 0
            
    # Write methods
    def write_n_observations(self, fname):
        sum_obs_xr = create_xr_dataset(self.sum_obs, self.lon_lat, os.path.basename(fname))
        sum_obs_xr.to_netcdf(fname)
        
    def write_sum_trucks(self, fname):
        sum_xr = create_xr_dataset(period.sum_trucks, period.lon_lat, os.path.basename(fname))
        sum_xr.to_netcdf(fname)
    
    def write_mean_trucks(self, fname):
        mean_xr = create_xr_dataset(self.mean_trucks, self.lon_lat, os.path.basename(fname))
        mean_xr.to_netcdf(fname)
        
    def write_trucks_vec(self, points, fname, fname_placeholder):
        got_points = points is not None and len(points) > 0
        if got_points:
            points.to_file(fname, driver = vec_driver_from_ext("." + fname.split(".")[1]))
        else:
            # write txt as placeholder
            with open(fname_placeholder, "w") as file:
                file.write("%s has length: %s. Nothing to write" %(os.path.basename(fname_placeholder), str(len(points))))
        
    def wrap_period(self, fnames):
        crs = EPSG_4326()
        ind_right = "index_right"
        self.write_n_observations(fnames[N_OBS()])
        self.mean_truck_n()
        self.mask_osm_to_trucks(self.osm_mask) # mask to trucks
        self.mask_sum_obs_to_trucks() # mask to trucks
        # merge n observations, n trucks and mean trucks into single points layer
        try:
            n_obs_points = raster_to_points(self.sum_obs, self.lon_lat, "sum_observations", crs)
            sum_trucks_points = raster_to_points(self.sum_trucks, self.lon_lat, "sum_trucks_sub_period", crs)
            sum_trucks_obs = gpd.sjoin(sum_trucks_points, n_obs_points, how="inner", op="intersects")
            sum_trucks_obs = sum_trucks_obs.drop([ind_right], axis=1)
            osm_points = raster_to_points(self.osm_mask, self.lon_lat, "osm_value", crs)
            osm_points["osm_name"] = osm_values_to_name(osm_points["osm_value"])
            sum_trucks_obs_osm = gpd.sjoin(sum_trucks_obs, osm_points, how="inner", op="intersects")
            sum_trucks_obs_osm = sum_trucks_obs_osm.drop([ind_right], axis=1)
            mean_trucks_points = raster_to_points(self.mean_trucks, self.lon_lat, "mean_trucks", crs)
            self.trucks_points_result = gpd.sjoin(sum_trucks_obs_osm, mean_trucks_points, how="inner", op="intersects")
            self.trucks_points_result = self.trucks_points_result.drop([ind_right], axis=1)
            self.trucks_points_result[VALID_PIXELS()] = [self.percentage_valid] * len(self.trucks_points_result)
            self.write_trucks_vec(self.trucks_points_result, fnames[TRUCKS_VEC()], fnames[TRUCKS_VEC_PHR()])
        except:
            Warning("No results could be written, detections could be empty")

#### AcquisitionProcessor

In [24]:
# AcquisitionProcessor processes all valid pixels of a single acquisition in cube
class AcquisitionProcessor():
    def __init__(self, date_np64, cube):
        self.date_np64 = date_np64
        self.cube = cube
        self.band_stack = cube.sel(time = date_np64)
        self.detector = None
        self.no_clouds = None
        self.osm_mask = None
        self.has_obs = None
        self.improves_cov = False
           
    def mask_clouds(self, cloud_masking_thresholds):       
        scl = MaskSet(self.band_stack.SCL)
        high_prob = scl.clouds_high_probability
        med_prob = scl.clouds_medium_probability
        cirrus = scl.cirrus
        no_data = scl.no_data
        rgb_cloud_mask = calc_rgb_cloud_mask(self.band_stack, cloud_masking_thresholds)
        self.no_clouds = (high_prob + med_prob + cirrus + no_data + rgb_cloud_mask) == 0
        self.band_stack = self.band_stack.where(self.no_clouds)
        
    # percentage Float secifies the percentage of valid observations
    # that is needed to be considered as valid acquisition
    def has_observations(self, minimum_valid_percentage):
        values = self.no_clouds.values
        # mask valid pixel mask to OSM roads
        values[self.osm_mask.roadmask == 0] = 0
        n_vals = np.count_nonzero(self.osm_mask.roadmask)
        n_valid = np.count_nonzero(values)
        percent_valid = (n_valid / n_vals) * 100
        self.has_obs = percent_valid >= minimum_valid_percentage
        
    def improves_coverage(self, period, min_improvement):
        first_acq = period.sum_obs is None
        if first_acq:
            period.sum_obs = np.zeros((self.band_stack.B02.shape[0], self.band_stack.B02.shape[1]))
        sum_obs_copy = period.sum_obs.copy() # get copy because original will be altered temporarilly
        cov_before = period.percentage_valid
        period.add_n_observations(self.no_clouds)
        period.percentage_covered()
        cov_after = period.percentage_valid
        period.sum_obs = sum_obs_copy # re-assign non-altered sum_obs array
        self.improves_cov = (cov_after - cov_before) > min_improvement
    
    def mask_with_osm(self, osm_mask):
        self.osm_mask = osm_mask
        self.band_stack = self.band_stack.where(self.osm_mask.roadmask != 0)
                
    def do_detection(self, thresholds):
        self.detector = TruckDetector(self.band_stack)
        self.detector.calc_no_trucks(thresholds)
        # check if enough pixel left
        self.detector.detect_trucks(thresholds)
        self.detector.filter_trucks()
        
    def write_detections(self, fname):
        trucks_xr = create_xr_dataset(self.detector.trucks, {"lat":self.cube.lat.values, "lon":self.cube.lon.values},
                                    os.path.basename(fname))
        # add no_clouds layer (n observations) for possible reload
        trucks_xr_with_no_clouds = trucks_xr.assign({"no_clouds":self.no_clouds})
        trucks_xr_with_no_clouds.to_netcdf(fname)
        
    def read_detections(self, fname):
        dataset = xr.open_dataset(fname)
        self.no_clouds = dataset["no_clouds"]
        trucks = dataset[os.path.basename(fname)]
        self.detector = TruckDetector(None)
        self.detector.trucks = trucks

__________
____________
## 7 | Execute processing
Process by grid __box__ (bbox_id), __sub-period__, __acquisition__

#### Main process

In [ ]:
# make or read processing grid
if os.path.exists(files["proc_grid"]):
    try:
        grid_gadm = gpd.read_file(files["proc_grid"])
    except:
        raise Exception("Failed reading proc grid from: " + files["proc_grid"])
else:
    grid_gadm = make_grid(grid_spacing, files)
if country is not None:
    grid_gadm = grid_gadm[grid_gadm["NAME_0"]==country]
grid_gadm = grid_gadm[grid_gadm["NAME_0"]!="United Kingdom"]
# calc temporal bounds of baseline sub-periods
if len(baseline_years) > 0:
    periods = yearly_period_from_target(target, baseline_years, timestamps_sub_period)
else:
    periods = calc_periods(n_days_sub, baseline, target)
sep = "-" * 80
print(sep)
trace = []
ext_arr = NC_EXT()
ext_vec = GPKG_EXT()
n_boxes = len(grid_gadm)
process_ids = [745, 746, 747, 748, 789, 790, 791, 792, 834, 836, 838, 877, 879, 881, 699, 701, 702, 703]#########
n_boxes = len(process_ids)##############
start_msg(weekdays, n_boxes, periods, timestamps_sub_period, osm_values, baseline_years, minimum_valid_observations, overwrite_results)
for i in range(n_boxes):
    i = list(grid_gadm.bbox_id).index(process_ids[i]) ##############################
    bbox = list(grid_gadm.geometry)[i].bounds
    bbox_id = list(grid_gadm[BBOX_ID()])[i]
    bbox_id_str = str(bbox_id)
    first, last = periods["first"], periods["last"]
    print("%s\n\nProcessing: %s\nbbox_id: %s\n\n%s" %(sep, str(i), bbox_id_str, sep))
    try:
        file_osm = get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dirs["ancil_roads"])
        # retry
        if not os.path.exists(file_osm): file_osm = get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dirs["ancil_roads"])
        osm = gpd.read_file(file_osm)
        # subset according to desired osm road levels (file might have been written with other values)
        value_not_desired = [v not in osm_values for v in list(osm["osm_value"])]
        osm = osm.drop(osm[value_not_desired].index)
    except:
        msg = "Could not get OSM roads: " + bbox_id_str
        Warning(msg)
        trace.append(msg)
        #continue
    for start, end in zip(first, last):
        period_files = []
        tracker_start = datetime.now()
        ts = first.index(start)
        ts_str = str(ts)
        dir_ts = dirs["processed"] # write into output pool
        if not os.path.exists(dir_ts): os.mkdir(dir_ts)
        dirs_ts = make_dirs_ts(dir_ts)
        print("TS: %s Period Start: %s End: %s" %(ts_str, str(start), str(end)))
        # check if yet processed
        fnames = sub_period_fnames(dirs_ts, bbox_id, str(start.date()) + "_" + str(end.date()), ext_arr, ext_vec)
        exists = {}
        for key, f in fnames.items():
            exists[key] = os.path.exists(f) if not f[-3:]=="txt" else True       
        if not exists[TRUCKS_VEC()]: exists[TRUCKS_VEC()] = os.path.exists(fnames[TRUCKS_VEC_PHR()]) # placeholder might exist instead
        already_proc = "because already processed"
        osm_msg = "Calculating OSM mask\n" + sep
        skip = all(exists.values()) and not overwrite_results
        if skip:
            print("Skipping " + already_proc)
        else:
            print("Getting cube")
            period = PeriodProcessor(start, end, bbox, bbox_id)
            period.get_cube(dataset, bands, tile_size, spatial_res, day_bins)
            t1 = datetime.now() # track time cube is open to prevent timeout
            print(osm_msg)
            period.calc_osm_mask(osm)
            p = 0
            obs_full = False
            # order by weekday of target weekdays
            period_dates_sorted = []
            for w in weekdays:
                dates_w = period.dates[[is_weekday(period_date, w) for period_date in period.dates]].flatten()
                for d in range(len(dates_w)): period_dates_sorted.append(dates_w[d])
            period.dates = period_dates_sorted
            while p < len(period.dates) and not obs_full:
                period_date = period.dates[p]
                if (datetime.now()-t1).seconds > 1800: # if 30 minutes exceeded get cube again to prevent timeout
                    print("Getting cube again to prevent timeout")
                    period.get_cube(dataset, bands, tile_size, spatial_res, day_bins)
                    t1 = datetime.now()
                    print(osm_msg)
                    period.calc_osm_mask(osm)
                acquisition = AcquisitionProcessor(period_date, period.cube)
                date_str = str(period_date).replace(":","_")[0:-10]
                print("Date: %s" %(date_str))
                fname = fname_acquisition_trucks(dirs_ts, bbox_id, date_str, ext_arr)
                if os.path.exists(fname):
                    print("File already processed\n%s" %(sep))
                    acquisition.read_detections(fname)
                    print("Number of trucks: %s" %(str(np.count_nonzero(acquisition.detector.trucks))))
                    period.add_acquisition(acquisition, date_str)
                    period_files.append(fname)
                else:
                    try:
                        min_valid_str = str(minimum_valid_observations)
                        fname_placeholder = os.path.join(dirs_ts["acquisitions_trucks"], 
                                                         date_str + min_valid_str + "_" + bbox_id_str + 
                                                         "_placeholder.txt").replace("-","_")
                        can_have_obs = not os.path.exists(fname_placeholder)
                        if can_have_obs:
                            print("Calculating cloud mask\n%s" %(sep))
                            acquisition.mask_clouds(cloud_masking_thresholds)
                            acquisition.mask_with_osm(period.osm_mask) # mask to OSM roads
                            # check if enough observations to be considered
                            acquisition.has_observations(minimum_valid_observations)
                            if not acquisition.has_obs:
                                with open(fname_placeholder, "w") as f:
                                    f.write("%s not fulfilled for date %s" %(min_valid_str, date_str))
                        else:
                            has_obs = can_have_obs
                    except:
                        print("Cloud masking failed. Assuming not enough observations")
                        acquisition.has_obs = False
                    if acquisition.has_obs:
                        print("Processing acquisition: %s" %(date_str))
                        try:
                            acquisition.do_detection(thresholds) # truck detection
                            print("Writing")
                            acquisition.write_detections(fname)
                            print("Number of trucks: %s" %(str(np.count_nonzero(acquisition.detector.trucks))))
                            period.add_acquisition(acquisition, date_str)
                            period_files.append(fname)
                            print("Done")
                        except:
                            print("Failed: %s" %(date_str))
                        print("Done with acquisition: %s\n%s" %(date_str, sep))
                    else:
                        print("Skipping, valid obs. below %s %%\n%s" %(minimum_valid_observations, sep))
                if period.sum_obs is not None:
                    period.percentage_covered()
                    print("Percentage valid in mosaic: %s" %(str(period.percentage_valid)))
                    n_obs_time = period.obs_time(n_obs)
                    print("Percentage covered by at least %s observations: %s" %(str(n_obs), str(n_obs_time)))
                    obs_full = period.percentage_valid >= obs_full_threshold and n_obs_time >= obs_full_threshold
                p += 1
            if len(period_files) > 0:
                print("%s\nNumber of files: %s" %(sep, str(len(period_files))))
                # read all processed files from disk in order not to hold in memory during processing
                print("Aggregating sub-period")
                period.wrap_period(fnames)
            else:
                msg = "No acquisitions in period %s to %s. In bbox_id: %s" %(str(start), str(end), bbox_id_str)
                Warning(msg)
                trace.append(msg)
            print("%s\nProcessing sub-period took: %s minutes" %(sep, str((datetime.now()-tracker_start).seconds/60)))
        print("%s\nDone with period %s of bbox_id %s\n%s" %(sep, ts_str, bbox_id_str, sep))    
    print("%s\nDone with bbox_id: %s\n%s\n%s" %(sep, bbox_id_str, sep, sep))
print("%s\nDone with all requested bboxes\n%s\n%s" %(sep, sep, sep*2))

--------------------------------------------------------------------------------

Starting truck detection processing 
..................................................

Weekdays: ['tuesday', 'wednesday', 'thursday']
Number of grid cells to process: 18
Number of periods: 4
Timestamps sub-period: 1
OSM roads: ['motorway', 'trunk', 'primary']
Baseline years: [2017, 2018, 2019]
Minimum valid observations to consider acquisition: 15 %
Overwrite results: True
..................................................

--------------------------------------------------------------------------------

Processing: 29
bbox_id: 745

--------------------------------------------------------------------------------
TS: 0 Period Start: 2017-04-01 00:00:00 End: 2017-06-21 00:00:00
Getting cube
Calculating OSM mask
--------------------------------------------------------------------------------
Date: 2017-05-23T10_40_25
Skipping, valid obs. below 15 %
----------------------------------------------------------

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


--------------------------------------------------------------------------------
Processing sub-period took: 0.8666666666666667 minutes
--------------------------------------------------------------------------------
Done with period 0 of bbox_id 745
--------------------------------------------------------------------------------
TS: 1 Period Start: 2018-04-01 00:00:00 End: 2018-06-21 00:00:00
Getting cube
Calculating OSM mask
--------------------------------------------------------------------------------
Date: 2018-04-03T10_41_05
Skipping, valid obs. below 15 %
--------------------------------------------------------------------------------
Date: 2018-04-10T10_30_19
File already processed
--------------------------------------------------------------------------------
Number of trucks: 2021
Percentage valid in mosaic: 30.487989987599057
Percentage covered by at least 2 observations: 0.0
Date: 2018-05-08T10_40_25
File already processed
-------------------------------------------------

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


--------------------------------------------------------------------------------
Processing sub-period took: 1.4 minutes
--------------------------------------------------------------------------------
Done with period 1 of bbox_id 745
--------------------------------------------------------------------------------
TS: 2 Period Start: 2019-04-01 00:00:00 End: 2019-06-21 00:00:00
Getting cube
Calculating OSM mask
--------------------------------------------------------------------------------
Date: 2019-04-23T10_54_59
File already processed
--------------------------------------------------------------------------------
Number of trucks: 884
Percentage valid in mosaic: 34.2821056972738
Percentage covered by at least 2 observations: 0.0
Date: 2019-04-30T10_36_47
File already processed
--------------------------------------------------------------------------------
Number of trucks: 2364
Percentage valid in mosaic: 87.4477828660784
Percentage covered by at least 2 observations: 23.3965409

...

/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))


Number of trucks: 340


/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))


Done
Done with acquisition: 2017-04-20T10_34_54
--------------------------------------------------------------------------------
Percentage valid in mosaic: 99.91034834195537
Percentage covered by at least 2 observations: 83.49431208413822
Date: 2017-04-27T10_20_25
Calculating cloud mask
--------------------------------------------------------------------------------
Processing acquisition: 2017-04-27T10_20_25


/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in less
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in less
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))


#### Merge detections of different processing boxes

In [ ]:
all_files = glob(dirs_ts[TRUCKS_VEC()]+os.sep+"*.gpkg")
start_dates = [string_to_datetime(date_str.split("_")[9]) for date_str in all_files]
first = periods["first"]
last = periods["last"]
for start, end in zip(periods["first"], periods["last"]):
    print("Merging from %s to %s" %(start, end))
    points_period = []
    for f in all_files:
        if string_to_datetime(f.split("_")[9]) == start:
            try:
                points = gpd.read_file(f)
                points_period.append(points)
            except:
                Warning("Could not read: %s" %(file))
    if len(points_period) > 0:
        merged = pd.concat(points_period)
        fsep = "_"
        fname = TRUCKS_VEC_MERGED() + fsep + fsep + str(start.date()) + fsep + str(end.date()) + GPKG_EXT()
        merged[TRUCK_COUNT_NORM()] = merged[MEAN_TRUCKS()] * (100/merged[VALID_PIXELS()])
        merged.to_file(os.path.join(dirs_ts[TRUCKS_VEC_MERGED()], fname), driver = GPKG())
    else:
        print("No files. \nStart: %s\nEnd: %s" %(start, end))

results_years = baseline_years.copy()
results_years.append(2020)
merged_files = glob(dirs_ts[TRUCKS_VEC_MERGED()]+os.sep+"*.gpkg")
median_values = []
for year in results_years:
    points_year = []
    for file in merged_files:
        if "__" + str(year) in os.path.basename(file):
            points = gpd.read_file(file)
            points_year.append(points)
    n_elements = np.array([np.array(x[x["osm_value"]==1][TRUCK_COUNT_NORM()]).sum() for x in points_year])
    print("---")
    print(n_elements)
    print(str(year) + "  Median: " + str(np.median(n_elements)))
    print(str(year) + "  Mean: " + str(np.mean(n_elements)))
    merged = pd.concat(points_year)
    file_str = TRUCKS_VEC_MERGED() + "_" + str(year)
    merged.to_file(os.path.join(dirs_ts[TRUCKS_VEC_MERGED()], file_str + GPKG_EXT()), driver=GPKG())
    print(np.array(merged["sum_observations"]).max())